In [1]:
import json
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patch

In [29]:
PATH_TO_FLIR = Path('/Users/maciejdziubinski/Downloads/FLIR_ADAS_1_3/')

CATEGORY_MAP = {
    1: 0,
    2: 1,
    3: 2,
    17: 3,
}

In [30]:
ls {PATH_TO_FLIR / 'train'}

Annotated_thermal_8_bit/  thermal_16_bit/           thermal_annotations.json*
RGB/                      thermal_8_bit/


In [34]:
def parse_labels(data_part, thermal_image_type='thermal_8_bit', debug=False):
    assert data_part in ['train', 'val']
    
    with open(PATH_TO_FLIR / data_part / 'thermal_annotations.json') as inp:
        annotations = json.load(inp)
    
    if debug:
        fig, ax = plt.subplots()
        
    categories = set()
    summary_txt = open(PATH_TO_FLIR / f'{data_part}_flir.txt', 'w')
    path_to_labels = PATH_TO_FLIR / 'labels' / data_part
    path_to_labels.mkdir(parents=True, exist_ok=True)


    for image_id, image_spec in enumerate(annotations['images']):
        filepath = image_spec['file_name']
        if thermal_image_type not in filepath:
            continue

        image_width = image_spec['width']
        image_height = image_spec['height']
        summary_txt.write(f'./{data_part}/{filepath}\n')

        labels_txt = open(path_to_labels / filepath.split('/')[1].replace('jpeg', 'txt'), 'w')

        if debug: 
            image = cv2.imread(str(PATH_TO_FLIR / data_part / filepath))
            ax.imshow(image)

        for annotation in annotations['annotations']:
            if annotation['image_id'] != image_id:
                continue

            x, y, width, height = annotation['bbox']

            category_id = CATEGORY_MAP[annotation['category_id']]
            if category_id not in categories:
                print(f'Adding the {category_id} category')
                categories.add(category_id)

            x_center = x + width / 2.
            y_center = y + height / 2.

            labels_txt.write(f'{category_id} {x_center / image_width} {y_center / image_height} {width / image_width} {height / image_height}\n')

            if debug:
                rect = patch.Rectangle((x, y), width, height, linewidth=1, edgecolor='r', facecolor='none')
                ax.add_patch(rect)
                center = patch.Rectangle((x_center, y_center), 1, 1, linewidth=4, edgecolor='g', facecolor='none')
                ax.add_patch(center)

        labels_txt.close()

        if debug: break

    if debug:
        fig.set_size_inches(20, 20)
        plt.show()

    summary_txt.close()

In [35]:
%time parse_labels('train')

Adding the 2 category
Adding the 0 category
Adding the 1 category
Adding the 3 category
CPU times: user 1min 14s, sys: 1.89 s, total: 1min 16s
Wall time: 1min 16s


In [36]:
%time parse_labels('val')

Adding the 0 category
Adding the 2 category
Adding the 1 category
Adding the 3 category
CPU times: user 2.47 s, sys: 316 ms, total: 2.78 s
Wall time: 2.8 s


In [37]:
cat /Users/maciejdziubinski/Downloads/FLIR_ADAS_1_3/labels/train/FLIR_00001.txt

2 0.41875 0.490234375 0.20625 0.23828125
2 0.7015625 0.4521484375 0.1375 0.150390625
2 0.1625 0.45703125 0.075 0.0546875
2 0.2234375 0.4609375 0.04375 0.06640625


In [38]:
!head /Users/maciejdziubinski/Downloads/FLIR_ADAS_1_3/train_flir.txt

./train/thermal_8_bit/FLIR_00001.jpeg
./train/thermal_8_bit/FLIR_00002.jpeg
./train/thermal_8_bit/FLIR_00003.jpeg
./train/thermal_8_bit/FLIR_00004.jpeg
./train/thermal_8_bit/FLIR_00005.jpeg
./train/thermal_8_bit/FLIR_00006.jpeg
./train/thermal_8_bit/FLIR_00007.jpeg
./train/thermal_8_bit/FLIR_00008.jpeg
./train/thermal_8_bit/FLIR_00009.jpeg
./train/thermal_8_bit/FLIR_00010.jpeg
